<a href="https://colab.research.google.com/github/taesangeom/SK-T-Academy/blob/main/DL/CNN/cnn_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# colab에서 할 때는 optuna를 설치해야 합니다.
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import optuna
import os
import time

In [ ]:
import tensorflow as tf

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_y), ( test_X, test_y) = fashion_mnist.load_data()
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
# 전처리
train_X = train_X /255.0
test_X  = test_X / 255.0

print(train_X.shape)
print(train_X[0].shape)
#train_X = train_X.reshape(60000, 28, 28,1)
train_X = train_X.reshape(-1, 28, 28,1) # 3d -- >4d
test_X = test_X.reshape(-1, 28,28,1)
print(train_X.shape)
print(train_X[0].shape)

(60000, 28, 28)
(28, 28)
(60000, 28, 28, 1)
(28, 28, 1)


In [ ]:
# 코드가 조금 길어지니까...모듈을 간단히 불러오겠습니다.
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [ ]:
model_vgg_fashion = tf.keras.Sequential(
    [
        # 원본 입력을 기준으로 설계 : 224 224 RGB --> (224,224,3)
        # ==> 28 , 28 , 1
        Conv2D( input_shape=(28,28,1), kernel_size=(3,3),
               filters = 64, padding="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters = 64, padding="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters = 128, padding="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters = 128, padding="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        Conv2D(  kernel_size=(3,3),
               filters = 256, padding="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters = 256, padding="same", activation="relu"),
        Conv2D(  kernel_size=(3,3),
               filters = 256, padding="same", activation="relu"),
        MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),

        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # Conv2D(  kernel_size=(3,3),
        #        filters = 512, padding="same", activation="relu"),
        # MaxPool2D( pool_size=(2,2), strides=(2,2)),
        ##====> 기본적인 이미지가 가지고 있는 특징 추출!!!!

        # 분류를 위한 NN
        # 1) 특징 추출 네트워크와 분류 네트워크 연결 : Flatten
        Flatten(),
        # 2) 분류를 위한 HL
        Dense( units = 4096, activation="relu"),
        Dense( units = 4096, activation="relu"),
        # 3) 출력용으로 대회가 1000가지 분류.,...
        Dense( units = 10, activation="softmax"),
    ]
)
model_vgg_fashion

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<Sequential name=sequential, built=True>

In [ ]:
# 위의 간단한 구조를 바탕으로 좀 더 디테일한 구조들에 대해서 실험!!!
# 1. 내가 무엇을 보고 최적화를 할지 함수로 설정!!!!
#    ML : 단순 파라미터들의 조합을 가지고 실험!!!!!
#    DL : 모데르이 구조에 대한 파라미터들을 가지고 실험!!!!
#         + 노드의 수, lr, AF etc
# ===> 내가 하려는 것에 맞는 함수를 직접 만들어서 사용을 해야 함!!!!!!

In [ ]:
# 참고 : 지극히 개인적인 실험 방식!!!

In [ ]:
# Step1) optuna가 주는 파라미터에 맞는 모델 구조를 생성하는 함수!!!!
# ===> 입력
#  1) 1번 conv layer에서 몇 가의 conv을 진행할지 실험(2,3) +
#  2) 2번 conv layer에서 몇 개의 conv을 진행할지 실험,,,
#  3) dropout 같은 구조에 있어서도,,rate 테스트,,,
#  4) 분류에 대한 Dense에서,,,node수를 조정하고 싶다!!!
# etc....
# ==> 실험을 하고자 하는 방향대로 만들어서 사용하면 됨!!!!!
# : 모델의 구조를 만들어 주는 함수!!! ( oputna의 obj와는 다른 함수!!!!!)

In [ ]:
def create_model( num_layers_conv1, num_filters_conv1,
                 num_layers_conv2, num_filters_conv2,
                  dnn_dropout_rate, dnn_num_node):
    # 여기서는 모델들을 설계하는 방법에 대한 파라미터들을 받아서 생성!!!
    # VGG가 적층 구조 방식이여서 layer들을 모델 구조에 append 하는 방식!!!
    # 참고 : 모델 구조가 적층 방식이 아니면, in/out의 구조로 모델을 작성해야 함!!!
    model = tf.keras.Sequential()
    # 1번 conv block
    model.add( Conv2D( input_shape=(28,28,1), kernel_size=(3,3),
               filters = 32, padding="same", activation="relu") )
    for i in range(num_layers_conv1): # 숫자만큰 conv 레이어 추가!!!
        model.add( Conv2D(  kernel_size=(3,3),
               filters = num_filters_conv1,
                    padding="same", activation="relu"))
    model.add( MaxPool2D( pool_size=(2,2), strides=(2,2)) )
    # ----------------------------

    # 2번 conv block
    model.add( Conv2D(  kernel_size=(3,3),
               filters = 64, padding="same", activation="relu") )
    for i in range(num_layers_conv2): # 숫자만큰 conv 레이어 추가!!!
        model.add( Conv2D(  kernel_size=(3,3),
               filters = num_filters_conv2,
                    padding="same", activation="relu"))
    model.add( MaxPool2D( pool_size=(2,2), strides=(2,2)) )

    # 분류를 위한 모델 설계
    model.add( Flatten())
    model.add( Dense( units =dnn_num_node , activation="relu") )
    model.add( Dropout( rate = dnn_dropout_rate ))
    model.add( Dense( units =dnn_num_node , activation="relu") )
    model.add( Dropout( rate = dnn_dropout_rate ))
    # ==> hidden layer의 수도 변수화 가능함!!!!!

    # 최종 출력용 ==> 고정!!!!!!!!
    model.add( Dense( units = 10, activation="softmax"))

    # 함수의 목적 : 출력물 --> 모델 구조도!!!
    return model


In [ ]:
from tensorflow import keras

In [ ]:
# Step2) 최적의 값을 찾는 objective function 설계!!!
# ==> ML과 유사하게 생성하면 됨!!!!
def objective_vgg( trail ):
    # 혹시 뒤에 keras 백단에서 살아있는 경우가 있음...
    keras.backend.clear_session()
    # ---> GPU의 메모리나 이런 부분이 클리어가 안 되는 경우가 있음!!!

    # 내가 테스트할 파라미터들을 제안 받는 것들에 대한 세팅!!!!
    num_layers_conv1=trail.suggest_int("num_layers_conv1", 1,5)
    num_filters_conv1=trail.suggest_int("num_filters_conv1", 32 , 256)
    num_layers_conv2= trail.suggest_int("num_layers_conv2", 1,5)
    num_filters_conv2=trail.suggest_int("num_filters_conv2", 128 , 512)
    dnn_dropout_rate=trail.suggest_float("dnn_dropout_rate", 0.1, 0.3 )
    dnn_num_node=trail.suggest_int("dnn_num_node", 256, 4096)
    # ++++ 본인이 앞에서 세팅한 모델 구조 함수에 대한 파라미터에 대한 제안!!!
    # ---------------------------------------------------------------------


    # +++ 모델의 학습하는 영향을 미치는 파라미터도 중요함!!!
    # learning rate에 관련된 부분이 중요함!!!!!
    # + batch_size, eopoch etc,,,,,,
    optimizer = trail.suggest_categorical( "optimizer", ["adam","sgd"])
    # ==> 최적화 방식도 실험으로 할 수 있고,,,,lr조절을 하면 됨!!!!!

    model = create_model( num_layers_conv1, num_filters_conv1,
                 num_layers_conv2, num_filters_conv2,
                  dnn_dropout_rate, dnn_num_node)
    model.compile(
        optimizer= optimizer,
        loss = "sparse_categorical_crossentropy",
        metrics = ["accuracy"]
    )
    # ---> 제안 받은 모델의 구조 & 어떻게 학습할지에 대한 세팅!!!!!

    # + 시간 관계상 수업에서는 callback은 생략!!!
    history = model.fit( train_X, train_y,
                        epochs = 2, # 시간 관계상 줄이겠습니다!!!!!
                        validation_split = 0.25,
                        batch_size = 1024,
    # +++ callback
    #callbacks = [cp_callback,es_callback ]
    )

    # object function 타켓!!!!!!
    # ==> 주관적으로 본인이 선택을 하시면 됨!!!!!!!!!!
    # 저는 여기서는 val_accuracy
    score = history.history["val_accuracy"][-1]
    return score

In [ ]:
# Step3)  optuna를 통한 실험!!!
study_my_vgg = optuna.create_study( direction="maximize")
# 왜? 내가 objective fuinction을 val_accuracy로 했기에.....
study_my_vgg.optimize( objective_vgg, n_trials=2, n_jobs=-1)

[I 2024-10-21 01:59:03,694] A new study created in memory with name: no-name-d32ba7e5-4678-4bdd-9133-9f30c9bbc92f
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
Epoch 1/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 279s 5s/step - accuracy: 0.1394 - loss: 2.3017 - val_accuracy: 0.2063 - val_loss: 2.2991
Epoch 2/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 283s 5s/step - accuracy: 0.1926 - loss: 2.3017 - val_accuracy: 0.2818 - val_loss: 2.2997
Epoch 2/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.2753 - loss: 2.2994 - val_accuracy: 0.3355 - val_loss: 2.2975


[I 2024-10-21 02:06:19,205] Trial 0 finished with value: 0.3355333209037781 and parameters: {'num_layers_conv1': 5, 'num_filters_conv1': 217, 'num_layers_conv2': 3, 'num_filters_conv2': 323, 'dnn_dropout_rate': 0.2352554208756247, 'dnn_num_node': 632, 'optimizer': 'sgd'}. Best is trial 0 with value: 0.3355333209037781.


44/44 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.2311 - loss: 2.2990 - val_accuracy: 0.2737 - val_loss: 2.2965


[I 2024-10-21 02:06:51,102] Trial 1 finished with value: 0.27373334765434265 and parameters: {'num_layers_conv1': 1, 'num_filters_conv1': 103, 'num_layers_conv2': 5, 'num_filters_conv2': 148, 'dnn_dropout_rate': 0.2568319779055963, 'dnn_num_node': 759, 'optimizer': 'sgd'}. Best is trial 0 with value: 0.3355333209037781.


In [ ]:
### 일반적으로는 epoch를 늘려서 하고, ES을 통해서 stop+ CP
# ===> 이것으로만 하기에는 조금 애매한 부분들이 있어서,,,
#      보통은 정말로 최적화가 필요할 때..( 기본 구조가 정해져있는 상태!! )
#      대회 출전 or 논문 제출할 때 수치 올리기 위해서 체크!!!!!
# +++ 최신 경향을 먼저( 기본 구조에 대해서 먼저 체크!! )
#     그 이후에 시간이 남거나, 투자해서 최적화를 해야한다면,,,시도!!!!!!